traffic signs tutorial: https://chsasank.github.io/keras-tutorial.html <br/>
what is that pet: https://www.udemy.com/deeplearning/learn/lecture/6798970#overview <br/>
traffic signs on medium tutorial: https://towardsdatascience.com/recognizing-traffic-signs-with-over-98-accuracy-using-deep-learning-86737aedc2ab

# Description
---

- The problem I am tackling in this assignment is The German Traffic Sign Recognition Benchamark ([GTSRB](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news)).
- The German Traffic Sign Benchmark is a multi-class, single-image classification challenge held at the International Joint Conference on Neural Networks (IJCNN) 2011. We cordially invite researchers from relevant fields to participate: The competition is designed to allow for participation without special domain knowledge. Our benchmark has the following properties:

    - Single-image, multi-class classification problem
    - More than 40 classes
    - More than 50,000 images in total
    - Large, lifelike database


## Tackling the problem and brainstorming
---

My approach to solve this problem will be a CNN as it's very successful and most commonly applied to analyzing visual imagery.  
- The tools will be used:
    - Keras
    - Convelutional Neural Network(CNN)
 


# Data Preprocessing
---

The dataset was donwloaded from the [GTSRB](https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/published-archive.html) website.

### Data normalization

- As I the images vary in size and color lets do [histogram equalization](https://en.wikipedia.org/wiki/Histogram_equalization) in `HSV` color space and resize the images to a standard size. **This method usually increases the global contrast of many images, especially when the usable data of the image is represented by close contrast values.**

- HSV = rgb2hsv( RGB ) converts the red, green, and blue values of an RGB image to hue, saturation, and value (HSV) values of an HSV image.

## Loading Data
---

In [52]:
import numpy as np
from skimage import io, color, exposure, transform
from sklearn.model_selection import train_test_split
import os
import glob
import h5py
from time import time
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization #, Merge
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.regularizers import l2


from keras import backend as K
import numpy as np


from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


NUM_CLASSES = 43
IMG_SIZE = 48

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
# Preprocessing

In [54]:
def preprocess_img(img):
    # Histogram normalization in y
#     hsv = color.rgb2hsv(img)
#     hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
#     img = color.hsv2rgb(hsv)
    # central scrop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,
              centre[1]-min_side//2:centre[1]+min_side//2,
              :]
    
    # rescale to standard size  
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))
    # 注意对于tensorflow和theano通道顺序的不同
    # roll color axis to axis 0
    #img = np.rollaxis(img,-1)

    return img


def get_class(img_path):
    return int(img_path.split('/')[-2])

## Preprocess all training images into a numpy array

In [55]:
try:
    with  h5py.File('X.h5') as hf: 
        X, Y = hf['imgs'][:], hf['labels'][:]
    print("Loaded images from X.h5")
    
except (IOError,OSError, KeyError):  
    print("Error in reading X.h5. Processing all images...")
    root_dir = 'data/Final_Training/Images/'
    imgs = []
    labels = []

    all_img_paths = glob.glob(os.path.join(root_dir, '*/*.ppm'))
    #打乱图片路径顺序
    np.random.shuffle(all_img_paths)
    for img_path in all_img_paths:
        try:
            img = preprocess_img(io.imread(img_path))
            
            # io.imread 读入的数据是 uint8
            
            label = get_class(img_path)
            imgs.append(img)
            labels.append(label)

            if len(imgs)%1000 == 0: print("Processed {}/{}".format(len(imgs), len(all_img_paths)))
        except (IOError, OSError):
            print('missed', img_path)
            pass

    X = np.array(imgs, dtype='float32')
    Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]
    # Y = ***[labels] 生成one-hot编码的方式
    with h5py.File('X.h5','w') as hf:
        hf.create_dataset('imgs', data=X)
        hf.create_dataset('labels', data=Y)

Loaded images from X.h5


## Load and Preprocess Test images

In [56]:
try:
    with h5py.File('X_test.h5') as hf: 
        X_test, y_test = hf['imgs'][:], hf['labels'][:]
    print("Loaded images from X_test.h5")
except (IOError,OSError, KeyError):  
    print("Error in reading X.h5. Processing all images...")
    import pandas as pd
    test = pd.read_csv('data/Final_Test/Images/GT-final_test.csv',sep=';')

    
    X_test = []
    y_test = []
    i = 0
    for file_name, class_id  in zip(list(test['Filename']), list(test['ClassId'])):
        img_path = os.path.join('data/Final_Test/Images/',file_name)
        X_test.append(preprocess_img(io.imread(img_path)))
        y_test.append(class_id)

    X_test = np.array(X_test, dtype='float32')
    y_test = np.array(y_test, dtype='uint8')

    with h5py.File('X_test.h5','w') as hf:
        hf.create_dataset('imgs', data=X_test)
        hf.create_dataset('labels', data=y_test)

Loaded images from X_test.h5


## Reshaping the data

In [69]:
index=np.zeros(1307, dtype='int')
for i in range(1307):
    index[i]=i*30+np.random.randint(0,30) 

X_validation = X[index]
y_validation = Y[index]
# X_val.shape
# create the training index1
index1=np.setdiff1d(np.array(range(39209)), index, assume_unique=True)
X_train=X[index1]
y_train=Y[index1]

normalize = 0
# Normalize the data: subtract the mean image
if normalize:
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image


print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_validation.shape)
print('Validation labels shape: ', y_validation.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (37902, 3, 48, 48)
Train labels shape:  (37902,)
Validation data shape:  (1307, 3, 48, 48)
Validation labels shape:  (1307,)
Test data shape:  (39209, 3, 48, 48)
Test labels shape:  (39209,)


### Loading Training Data

# Models
- Let’s now define our models. We’ll use feed forward network with 6 convolutional layers followed by a fully connected hidden layer. We’ll also use dropout layers in between. Dropout regularizes the networks, i.e. it prevents the network from overfitting.

- All our layers have relu activations except the output layer. Output layer uses softmax activation as it has to output the probability for each of the classes.

- Sequential is a keras container for linear stack of layers. Each of the layers in the model needs to know the input shape it should expect, but it is enough to specify input_shape for the first layer of the Sequential model. Rest of the layers do automatic shape inference.

- To attach a fully connected layer (aka dense layer) to a convolutional layer, we will have to reshape/flatten the output of the conv layer. This is achieved by Flatten layer

- Go through the documentation of keras (relevant documentation : here and here) to understand what parameters for each of the layers mean.